# Collect data

In [1]:
# import Lib
import pandas as pd
import numpy as np
import urllib.robotparser
import time
from bs4 import BeautifulSoup
import requests
from fractions import Fraction
import re

### Check dữ liệu trên trang này có hợp pháp khi thu thập

In [2]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://store.steampowered.com/robots.txt')
rp.read()

url = 'https://store.steampowered.com/search/?category1=998&ndl=1'
rp.can_fetch('*', url)

True

Dữ liệu hợp pháp!

### Crawl data với các thuộc tính

File steam_game.csv:
- title  
- genre  
- tags  
- category  
- is_dlc  
- is_mature  
- franchise  
- release date  
- developer  
- publisher  
- languages  
- overwhelmingly positive count  
- user reviews count  
- negative count  
- price

File reviews.csv:
- title
- found_helpful
- recommended
- reviews

In [3]:
# Title = []  
# Genre = []  
# Tags = []   
# withDLC = []  
# isMature = [] 
# Franchise = []
# ReleaseDate = []  
# Developer = []  
# Publisher = []
# Languages = []
# PositiveReviews = []
# TotalReviews = []
# NegativeReviews = []  
# OriginalPrice = []
# DiscountPercent = []

In [4]:
data = pd.DataFrame()
data_review = pd.DataFrame()

# data = pd.read_csv('steam_game.csv')
# data.tail()

In [5]:
def crawl_one_game_review(item_review_url, title):
    response = requests.get(item_review_url)
    content = response.text
    if response.status_code == 200:
        soup = BeautifulSoup(content, 'html.parser')
        
        # set all attribute is NaN
        Title = []
        FoundHelpful = []
        Recommended = []
        Reviews = []

        try:
            review_box = soup.find_all('div', attrs={'class': 'apphub_UserReviewCardContent'})

            for i in review_box:
                Title.append(title)

                Recommended.append(i.find('div',attrs={'class':'title'}).string.strip())

                review = i.find('div',attrs={'class':'apphub_CardTextContent'}).text.split('\n')[2].strip()
                # print(review)
                Reviews.append(review)

                found_helpful = i.find('div',attrs={'class':'found_helpful'}).text.split('\n')[1].strip()
                # print(found_helpful)
                FoundHelpful.append(found_helpful)
                
            crawled_data_review = pd.DataFrame({"Title": Title,
                                                 "Recommended": Recommended,
                                                 "Reviews": Reviews,
                                                 "FoundHelpful": FoundHelpful})
            
            global data_review
            data_review = pd.concat([data_review, crawled_data_review], axis = 0)
                
        except:
            pass


In [6]:
def crawl_one_game(item_url):
    response = requests.get(item_url)
    content = response.text
    if response.status_code == 200:
        soup = BeautifulSoup(content, 'html.parser')
        
        # set all attribute is NaN
        title = np.nan 
        genre = np.nan
        tags = np.nan
        with_dlc = np.nan 
        is_mature = np.nan
        franchise = np.nan
        release_date = np.nan
        developer = np.nan
        publisher = np.nan
        languages = np.nan
        positive_reviews = np.nan
        total_reviews = np.nan
        negative_reviews = np.nan

        # # Check if a dlc
        # is_dlc = soup.find('div',attrs={'class':'game_area_bubble game_area_dlc_bubble'})
        # # Check if a soundtrack
        # is_soundtrack = soup.find('div',attrs={'class':'game_area_bubble game_area_soundtrack_bubble'})
        
        # if is_dlc or is_soundtrack:
        #     return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews

        # # Check game has DLC
        # try:
        #     with_dlc_tmp = soup.find('div',attrs={'id':'gameAreaDLCSection'})
        #     with_dlc = (True if with_dlc_tmp else False) 
        # except:
        #     pass

        # Title
        try:
            title = soup.find('div',attrs={'class':'apphub_AppName'}).string.strip() 
        except:
            pass

        # # Genere
        # try:
        #     genre = [] # A game can have more than one genre
        #     genre_tmp = soup.find('div',attrs={'id':'genresAndManufacturer'}).find('span')
        #     for i in genre_tmp.find_all('a'):
        #         genre.append(i.text.strip())
                
        #     genre = ", ".join(genre)
        # except:
        #     genre = np.nan
        #     pass

        # # Tag
        # try:
        #     tags = [] # A game can have more than one tag
        #     for i in soup.find('div',attrs={'class':'glance_tags popular_tags'}).find_all('a'):         
        #         if i.text.strip() != '+':
        #             tags.append(i.text.strip())
                    
        #     tags = ", ".join(tags)
        # except:
        #     tags = np.nan
        #     pass

        # # Check Mature
        # try:
        #     is_mature_tmp = soup.find('div',attrs={'id':'game_area_content_descriptors'})
        #     is_mature = (True if is_mature_tmp else False)
        # except:
        #     pass

        # # Developer, Publisher, Franchise
        # try:
        #     for i in soup.find('div',attrs={'id':'genresAndManufacturer'}).find_all('div',attrs={'class':'dev_row'}):
        #         if i.find('b').text == 'Developer:':
        #             developer = i.find('a').text
        #         elif i.find('b').text == 'Publisher:':
        #             publisher = i.find('a').text
        #         elif i.find('b').text == 'Franchise:':
        #             franchise = i.find('a').text
        # except:
        #     pass

        # # Release Date
        # try:
        #     release_date = soup.find('div',attrs={'class':'date'}).string.strip()
        # except:
        #     pass 

        # # Languages
        # try:
        #     languages = [] # A game can have more than one language
        #     for i in soup.find('div',attrs={'id':'languageTable'}).find_all('td',attrs={'class':'ellipsis'}):   
        #         languages.append(i.text.strip())
            
        #     languages = ", ".join(languages)
        # except:
        #     languages = np.nan
        #     pass

        # # All review
        # try:
        #     total_reviews = soup.find('label',attrs={'for':'review_type_all'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
        #     total_reviews = int(total_reviews.replace(')','').replace('(',''))
        # except:
        #     pass

        # # Negative review
        # try:
        #     negative_reviews = soup.find('label',attrs={'for':'review_type_negative'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
        #     negative_reviews = int(negative_reviews.replace(')','').replace('(',''))
        # except:
        #     pass

        # # Positive review
        # try:
        #     positive_reviews = soup.find('label',attrs={'for':'review_type_positive'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
        #     positive_reviews = int(positive_reviews.replace(')','').replace('(',''))
        # except:
        #     pass
        
        # Crawl all reviews of game
        id_game = re.search(r'\b\d+\b', item_url).group(0)
        url_review = f'https://steamcommunity.com/app/{id_game}/reviews/?browsefilter=toprated'

        crawl_one_game_review(url_review, title)
 
        # Return 
        return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews


In [7]:
flag_page=True
index = 201

# get the games from all the pages and not give it a range because its very dynamic
while(flag_page and index!=401):
    Title = []  
    Genre = []  
    Tags = []   
    withDLC = []  
    isMature = [] 
    Franchise = []
    ReleaseDate = []  
    Developer = []  
    Publisher = []
    Languages = []
    PositiveReviews = []
    TotalReviews = []
    NegativeReviews = []  
    OriginalPrice = []
    DiscountPercent = []
    print(index)
    
    # Add page parameter
    parameter = '&page='+str(index)
    
    # Send a request to the website
    web_response = requests.get(url+parameter)
        
    # Get the content of the request
    web_content = web_response.text
    
    if web_response.status_code == 200: # request successfully
        web_soup = BeautifulSoup(web_content, 'html.parser')
        
        item_link = []
        
        # Check if the last page
        if index != 1:
            # Find the number of buttons on the page and return False if it has 2
            button = web_soup.find_all('a',attrs={'class':'pagebtn'})
            if len(button)!= 2 :
                flag_page = False
    
        # Add each link to links
        links = web_soup.find('div', attrs={'id': 'search_resultsRows'})
        if links is None:
            index += 1
            continue
    
        for link in links.find_all('a'):
            # # Price with discount
            # with_discount = link.find('div',attrs={'class':'discount_block search_discount_block'})
            # # Price without discount
            # without_discount = link.find('div',attrs={'class':'discount_block search_discount_block no_discount'})
            # # Free price
            # free_price = link.find('div',attrs={'class':'discount_block no_discount search_discount_block'})
            
            # if with_discount:
            #     DiscountPercent.append(with_discount.find('div',attrs={'class':'discount_pct'}).text)
            #     OriginalPrice.append(with_discount.find('div',attrs={'class':'discount_original_price'}).text)      
            # elif without_discount:
            #     DiscountPercent.append(np.nan)
            #     OriginalPrice.append(without_discount.find('div',attrs={'class':'discount_final_price'}).text)
            # elif free_price:
            #     DiscountPercent.append(np.nan)
            #     OriginalPrice.append(free_price.find('div',attrs={'class':'discount_final_price free'}).text)
            # else:
            #     break
        
            item_link.append(link['href'])
            
        # Add attribute of each game
        for item in item_link:    
            title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews = crawl_one_game(item)              
            
            # Add values into attributes
    #         Title.append(title)  
    #         Genre.append(genre)
    #         Tags.append(tags)  
    #         withDLC.append(with_dlc)
    #         isMature.append(is_mature)
    #         Franchise.append(franchise)
    #         ReleaseDate.append(release_date) 
    #         Developer.append(developer)
    #         Publisher.append(publisher)
    #         Languages.append(languages)
    #         PositiveReviews.append(positive_reviews)
    #         TotalReviews.append(total_reviews)
    #         NegativeReviews.append(negative_reviews)
    # crawled_data = pd.DataFrame({"Title": Title,
    #                  "Genre": Genre,
    #                  "Tags": Tags,
    #                  "withDLC": withDLC,
    #                  "isMature": isMature,
    #                  "Franchise": Franchise,
    #                  "ReleaseDate": ReleaseDate,
    #                  "Developer": Developer,
    #                  "Publisher": Publisher,
    #                  "Languages": Languages,
    #                  "PositiveReviews": PositiveReviews,
    #                  "TotalReviews": TotalReviews,
    #                  "NegativeReviews": NegativeReviews,
    #                  "OriginalPrice": OriginalPrice,
    #                  "DiscountPercent": DiscountPercent})   
                     
    # data = pd.concat([data, crawled_data], axis = 0) 
    # Increase page index
    index += 1

201


202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
data

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,OriginalPrice,DiscountPercent
0,Baldur's Gate 3,"Adventure, RPG, Strategy","RPG, Choices Matter, Story Rich, Character Cus...",True,True,Baldur's Gate,"3 Aug, 2023",Larian Studios,Larian Studios,"English, French, German, Spanish - Spain, Poli...","(499,931)","(516,754)","(16,823)","990.000,00₫",-10%
1,Counter-Strike 2,"Action, Free to Play","FPS, Shooter, Multiplayer, Competitive, Action...",True,True,NaN,"21 Aug, 2012",Valve,Valve,"English, Czech, Danish, Dutch, Finnish, French...","(6,846,587)","(7,811,257)","(964,670)",Free,NaN
2,Lethal Company,"Action, Adventure, Indie, Early Access","Online Co-Op, Horror, First-Person, Psychologi...",False,True,NaN,"23 Oct, 2023",Zeekerss,Zeekerss,English,"(222,693)","(226,944)","(4,251)","142.000,00₫",NaN
3,Cyberpunk 2077,RPG,"Cyberpunk, Open World, Nudity, RPG, Singleplay...",True,True,NaN,"9 Dec, 2020",CD PROJEKT RED,CD PROJEKT RED,"English, French, Italian, German, Spanish - Sp...","(565,104)","(698,558)","(133,454)","990.000,00₫",-50%
4,ELDEN RING,"Action, RPG","Souls-like, Dark Fantasy, Open World, RPG, Dif...",False,True,Bandai Namco Entertainment,"24 Feb, 2022",FromSoftware Inc.,FromSoftware Inc.,"English, French, Italian, German, Spanish - Sp...","(688,881)","(745,747)","(56,866)","898.000,00₫",-40%
5,Dota 2,"Action, Strategy, Free to Play","Free to Play, MOBA, Multiplayer, Strategy, eSp...",True,False,Dota,"9 Jul, 2013",Valve,Valve,"English, Bulgarian, Czech, Danish, Dutch, Finn...","(1,759,065)","(2,143,012)","(383,947)",Free,NaN
6,Apex Legends™,"Action, Adventure, Free to Play","Free to Play, Battle Royale, Multiplayer, Shoo...",False,False,Apex Legends,"4 Nov, 2020",Respawn Entertainment,Electronic Arts,"English, French, Italian, German, Spanish - Sp...","(613,747)","(774,092)","(160,345)",Free,NaN


In [ ]:
data_review['Title']

0    Kinkoi: Golden Loveriche
1    Kinkoi: Golden Loveriche
2    Kinkoi: Golden Loveriche
3    Kinkoi: Golden Loveriche
4    Kinkoi: Golden Loveriche
               ...           
5              Maximum Action
6              Maximum Action
7              Maximum Action
8              Maximum Action
9              Maximum Action
Name: Title, Length: 6256, dtype: object

In [ ]:
data.to_csv('steam_game.csv', index = False)

In [1]:
data_review.to_csv('reviews.csv', index = False)

NameError: name 'data_review' is not defined